In [1]:
import sys
sys.path.append('..')
import numpy as np
from tiff_helpers import *


In [33]:
path = '/media/arimorcos/SSD 1/AM144/141204/AM144_001_001.tif'
pages = range(1, 2000, 2)
tiff = read_tiff(path, pages)

In [4]:
def mean_center_img(img):
    """
    Makes each image in a stack have zero mean and unit std
    :param img: 3d array of images
    :return: normalized image array
    """

    for page in range(img.shape[2]):
        temp_page = img[:, :, page]
        img[:, :, page] = (temp_page - temp_page.mean())/temp_page.std()

    return img

In [34]:
tiff.shape

(512, 512, 1000)

In [35]:
%%time
slow = mean_center_img(tiff)

CPU times: user 13.4 s, sys: 0 ns, total: 13.4 s
Wall time: 13.4 s


In [37]:
import time

In [45]:
from numba import jit

In [46]:
@jit
def mean_center_img_numba(img):
    """
    Makes each image in a stack have zero mean and unit std
    :param img: 3d array of images
    :return: normalized image array
    """    
    
    # Get original shape 
    orig_shape = img.shape
    
    # Get number of pixels per page 
    num_pixels_per_page = orig_shape[0]*orig_shape[1]
    
    # Reshape tiff to num_pixels_per_page x num_pages
    reshape_img = np.reshape(img, (num_pixels_per_page, -1))
    
    # Take mean and std 
    page_means = reshape_img.mean(axis=0)
    page_stds = reshape_img.std(axis=0)
    
    # Mean center each page 
    reshape_img = (reshape_img - page_means)/page_stds
    
    # Reshape to appropriate size 
    return np.reshape(reshape_img, orig_shape)  
    
    

In [43]:
def mean_center_img_fast(img):
    """
    Makes each image in a stack have zero mean and unit std
    :param img: 3d array of images
    :return: normalized image array
    """    
    
    # Get original shape 
    orig_shape = img.shape
    
    # Get number of pixels per page 
    num_pixels_per_page = orig_shape[0]*orig_shape[1]
    
    # Reshape tiff to num_pixels_per_page x num_pages
    reshape_img = np.reshape(img, (num_pixels_per_page, -1))
    
    t = time.time()
    
    # Take mean and std 
    page_means = reshape_img.mean(axis=0)
    page_stds = reshape_img.std(axis=0)
    
    print time.time() - t
    
    t = time.time()
    # Mean center each page 
    reshape_img = (reshape_img - page_means)/page_stds
    print time.time() - t
    
    # Reshape to appropriate size 
    return np.reshape(reshape_img, orig_shape)  
    
    

In [44]:
%%time
fast = mean_center_img_fast(tiff)

1.57793188095
1.78193497658
CPU times: user 2.66 s, sys: 740 ms, total: 3.4 s
Wall time: 3.4 s
